In [38]:
import os
import sys, os
import dill
import sqlalchemy
import ipywidgets as widgets
from aepsych.plotting import plot_strat
from aepsych.server import AEPsychServer
from IPython.display import display
from ipywidgets import HTML, AppLayout, Box, Button, Label, Layout, Text,GridspecLayout
from IPython.core.display import HTML as html

connect_out = widgets.Output(layout=Layout(padding="2px", height="150px"))

database_path = None
with connect_out:
    server = AEPsychServer()
    connect_out.clear_output()
strat = None

#---------- Style ----------

display(html("""<style>
.jp-CodeCell {
    width:100% !important;
    background: whitesmoke;
}
.jp-Notebook .jp-Cell {
    padding: 0 !important;
}

.jupyter-widgets.widget-tab {
    margin: 40px 30px;
}

.p-Collapse-header,
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current:before,
.jupyter-button ,
.widget-button,
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
    border: 1px solid #dcdcdc;
}


.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab{
border-top-right-radius:4px;
border-top-left-radius: 4px;
border: 1px solid #dcdcdc;
}
.p-Collapse-contents{
border-bottom-right-radius:4px;
border-bottom-left-radius: 4px;
border: 1px solid #dcdcdc;
border-top:none;
}

.p-Collapse-open > .p-Collapse-header {
border-bottom-right-radius:0px;
border-bottom-left-radius: 0px;
}
body.jp-Notebook {
    margin: 0 !important;
    padding: 0 !important;
    height: 100vh !important;
    background: whitesmoke;
}
.widget-inline-hbox {
    margin: 0px;
}
.widget-output {
   # padding: 10px !important;
   # background: white;
   # border-radius: 4px !important;
   # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # border: 1px solid #e3e3e3;
   # border-radius: 4px;
   # -webkit-box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # margin: 10px;
   # max-width: 600px; 
    height: auto !important;
    width: auto !important;
    overflow-y: auto;
}


.widget-text input[type="text"],
.widget-text input[type="number"],
.widget-dropdown > select {
   border-radius: 4px;
   box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
}

.widget-vbox {
    justify-content: center;
    width: 100%;
    width: -webkit-fill-available !important;
    display: flex;
    border-radius: 4px;
    background: white;
    height: fit-content;
    padding: 20px;
   # box-shadow: 0px 1px 3px rgb(0 0 0 / 12%), 0px 1px 2px rgb(0 0 0 / 24%);
}


[data-jp-theme-light='true'] .jp-RenderedImage img.jp-needs-light-background {
    width: 100%;
}

.jupyter-widgets.widget-tab {
    min-width: 30% !important;
}
html {
    background: #F2F2F2;
    height: 100%;
    margin: 0px;
    padding: 0px;

}

.widget-dropdown,
.widget-text{
    width: auto;

}
.widget-label {
    display: flex;
    align-items: center;
}
.jp-RenderedText[data-mime-type='application/vnd.jupyter.stderr'] {
    background: white;
    padding-top: var(--jp-code-padding);
}
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
    border-top-left-radius: 0px;
    border-top-right-radius: 0px;
}

</style>"""))


header = HTML(
    """
<div class="nav" style=" 
margin: 0px;
background-color: #F2F2F2;
font-weight: bold; 
box-shadow: 0px 1px 3px rgb(0 0 0 / 12%), 0px 1px 2px rgb(0 0 0 / 24%);
">
    <h1 style="
    text-align: center;
    font-family: helvetica;
    font-weight: 400;
    font-size: 20px;
    display: flex;
    align-items: center; 
    padding: 15px 20px;
    margin: 0px;
    hieght: 100px; 
    jusify-content: flex-start;
    color:rgb(80,103,132)">
    <img src="./logo.png" alt="logo" width="50" height="50" style="margin-right: 10px;" >
    AEPsych Visualizer
    </h1>
</div>
"""
)


input_style = {"description_width": "initial"}
input_layout= Layout(margin="5px 0")
btn_style = Layout(margin="10px",width="95%")

btn_box_layout = Layout(display="flex", justify_content="flex-end")
file_output = widgets.Output(
    layout={
            "overflow": "scroll", 
            "height": "50vh",
            "width": "100%",
            "padding": "5px"
           }
)

# ---------- Inputs -----------

def on_value_change(change):
    file_output.clear_output()
    with file_output:
        change["new"]
#         print(change['new'].keys())
#         print(change["new"])

        
def on_value_change_input(change):
    inputs_output.clear_output()
    with inputs_output:
        change["new"]


uploader = widgets.FileUpload(
    description="Select File",accept=".db", multiple=False, layout=btn_style)

yes_label = Text(
    value="Yes Trial", placeholder="yes_label",
    description="yes_label: ", disabled=False, style=input_style,
    layout=input_layout
)

no_label = Text(
    value="No Trial", placeholder="no_label", 
    description="no_label: ", disabled=False, style=input_style,
    layout=input_layout
)

target_level = widgets.BoundedFloatText(
    value=0.75, min=0, max=1, 
    step=0.1, description="target_level:", disabled=False, 
    style=input_style, layout=input_layout
)

cred_level = widgets.BoundedFloatText(
    value=0.95, min=0, max=1, step=0.1, 
    description="cred_level:", disabled=False, 
    style=input_style, layout=input_layout
)

xlabel = Text(
    value="Angle (degrees)", placeholder="x axis label",
    description="xlabel: ", disabled=False, style=input_style,
    layout=input_layout
)

ylabel = Text(
    value="Detection Probability", placeholder="y axis label", 
    description="ylabel: ", disabled=False, style=input_style,
    layout=input_layout
)

# Observes input changes
uploader.observe(on_value_change, names="value")

inputs_output = widgets.Output(
    layout={"border": "1px solid black", "padding": "20px", "margin": "20px", "min-height ": "100px"}
)
yes_label.observe(on_value_change_input, names="value")
no_label.observe(on_value_change_input, names="value")
target_level.observe(on_value_change_input, names="value")
cred_level.observe(on_value_change_input, names="value")
xlabel.observe(on_value_change_input, names="value")
ylabel.observe(on_value_change_input, names="value")


# -------- ports ----------
server_ip = widgets.Text(
    placeholder="0.0.0.0", # must use regex to check pattern and type int
    description='Server IP',
    disabled=False,
    layout=Layout(display='flex', flex_flow="column", margin="10px"))
port = widgets.Text(
    placeholder="5555", # must add regex, type check, and error handling
    description='Port',
    disabled=False,
    layout=Layout(display='flex', flex_flow="column", margin="10px", justify_content="center"))

connected_ip = HTML(f"Connected by client at {server_ip.value} {port.value}")

def on_change_server_input(change):
    with inputs_output:
        change["new"]

server_ip.observe(on_change_server_input, names="value")
port.observe(on_change_server_input, names="value") 

connection_inputs = widgets.Box([server_ip, port, connected_ip], 
                               layout=Layout(justify_content="flex-end",margin="4px"))
# -------- Logging --------

logs_style = Layout(width='600px', height='160px', border='solid', background="black")
logger_out = widgets.Output(layout=logs_style)

# -------- Config --------
# config_out = widgets.Output()

config_textarea = widgets.Textarea(layout=logs_style)
# params_label = widgets.Label(value="Parameters:")
# params_boxes = widgets.VBox([])

# def config_input(change):
#     print(change.new)
    
    
# config_textarea.observe(config_input, names=['value'])

# --------- Interactive ------------

# --> Tab 1 
param_one = widgets.FloatText(
    value=1.23,
    description='Parameter1: ',
    disabled=False,
    layout=Layout(margin="5px 10px")
)
param_two = widgets.FloatText(
    value=3.45,
    description='Parameter2: ',
    disabled=False,
    layout=Layout(margin="5px 10px")
)
outcome = widgets.IntText(value=0, description='Outcome: ', 
                          disabled=False,layout=Layout(margin="5px 10px"))

param_box_tab = widgets.VBox([param_one, param_two, outcome], 
                         layout=Layout(max_width="250px"))

# --> Tab 2
#  ------- Inputs ----------
file_output = widgets.Output()
inputs_output = widgets.Output()

def on_value_change(change):
    file_output.clear_output()
    with file_output:
        change["new"]
        print(change["new"])
        
        
def on_value_change_input(change):
    inputs_output.clear_output()
    with inputs_output:
        change["new"]
        print(change["new"])


uploader = widgets.FileUpload(
    description="Select File",accept=".db", multiple=False, layout=btn_style)

yes_label = Text(
    value="Yes Trial", placeholder="yes_label",
    description="yes_label: ", disabled=False, style=input_style,
    layout=input_layout
)

no_label = Text(
    value="No Trial", placeholder="no_label", 
    description="no_label: ", disabled=False, style=input_style,
    layout=input_layout
)

target_level = widgets.BoundedFloatText(
    value=0.75, min=0, max=1, 
    step=0.1, description="target_level:", disabled=False, 
    style=input_style, layout=input_layout
)

cred_level = widgets.BoundedFloatText(
    value=0.95, min=0, max=1, step=0.1, 
    description="cred_level:", disabled=False, 
    style=input_style, layout=input_layout
)

xlabel = Text(
    value="Angle (degrees)", placeholder="x axis label",
    description="xlabel: ", disabled=False, style=input_style,
    layout=input_layout
)

ylabel = Text(
    value="Detection Probability", placeholder="y axis label", 
    description="ylabel: ", disabled=False, style=input_style,
    layout=input_layout
)

# Observes input changes
uploader.observe(on_value_change, names="value")

inputs_output = widgets.Output(
    layout={"border": "1px solid black", "padding": "20px", "margin": "20px", "min-height ": "100px"}
)
yes_label.observe(on_value_change_input, names="value")
no_label.observe(on_value_change_input, names="value")
target_level.observe(on_value_change_input, names="value")
cred_level.observe(on_value_change_input, names="value")
xlabel.observe(on_value_change_input, names="value")
ylabel.observe(on_value_change_input, names="value")

# --------- Accordion -----------
accordion = widgets.Accordion(
    children=[
        widgets.VBox([no_label, yes_label], layout=Layout(padding="5px")),
        widgets.VBox(
            [
                target_level,
                cred_level,
            ],
            layout=Layout(padding="5px")
        ),
        widgets.VBox([xlabel, ylabel],layout=Layout(padding="5px")),
    ]
)
accordion.set_title(0, "Outcome Labels")
accordion.set_title(1, "Parameters")
accordion.set_title(2, "Axis Labels")


button_submit = widgets.Button(
    description="Submit", disabled=False, 
    button_style="success", tooltip="submit", 
    layout=btn_style
)

def submit():
    file_output.clear_output()
    with file_output:
        display_plot()
        
        
def submit_clicked(b):
    submit()
    
    
button_submit.on_click(submit_clicked)

update_model_tab = widgets.VBox([accordion,
            widgets.Box([button_submit], layout=btn_box_layout),
            file_output])

# --> Tab 3


tab_labels = ['New Parameters', 'Update Model', 'Send to Client']
tab_widgets = [param_box_tab, update_model_tab, Label('Send to Client')]

children = [widget for widget in tab_widgets]
interactive_tabs = widgets.Tab(layout=Layout(width='100%'))
interactive_tabs.children = children
for i in range(len(children)):
    interactive_tabs.set_title(i, tab_labels[i])

    
# ------- View 2 --------
plot_ouput = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))

def display_plot():
    global strat
    plot_ouput.clear_output()
    with plot_ouput:
# #------- Error Handling -----
#     error = False
#     with plot_ouput:
#         plot_ouput.clear_output()
#         if len(ylabel.value) > 35: 
#             error = True
# #             raise Exception("ylabel must be less than 35 characters ")
#             print("ylabel must be less than 35 characters ")
#         elif len(xlabel.value) > 35: 
#             error = True
#             print("xlabel must be less than 35 characters ")
#         elif len(yes_label.value) > 35: 
#             error = True
#             print("yes_label must be less than 35 characters ")
#         elif len(no_label.value) > 35: 
#             error = True
#             print("no_label must be less than 35 characters ")

#     if error == False: 
#         inputs_output.clear_output()
        if strat != None:
            plot_strat(
                strat,
                xlabel=xlabel.value,
                ylabel=ylabel.value,
                no_label=no_label.value,
                yes_label=yes_label.value,
                cred_level=cred_level.value,
                target_level=target_level.value,
            )
            return True 
        else:
            print("Server has no experiment records!")
            return False 

out = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))


exp_details = widgets.Button(
    description='Exp Details',
    disabled=False,
    button_style='',
    tooltip='Click me')
plot_view = widgets.Button(
    description="Plot View",
    disable=False,
    button_style="",
    tooltip="Click me")
table_view = widgets.Button(
    description="Table View",
    disable=False,
    button_style="",
    tooltip="Click me")
logs = widgets.Button(
    description="Logs",
    disable=False,
    button_style="",
    tooltip="Click me")
interactive = widgets.Button(
    description="Interactive",
    disable=False,
    button_style="",
    tooltip="Click me")

navigation = widgets.Box([exp_details,plot_view,table_view,logs,interactive],
layout=Layout(  
        display='flex',
        flex_flow="column",
        width='200px',
        margin="0 2.5%",
        min_width="152px"
        ))
output_header_style = Layout(justify_content="center")
nav_display_style = Layout(display="flex",
        flex_flow="column")

container_one = widgets.Box([
    Label("Exp Details", layout=output_header_style), config_textarea],
    layout=nav_display_style )
container_two = widgets.Box([
    Label("Plot View", layout=output_header_style), plot_ouput],
    layout=Layout(max_width="600px",display="flex",flex_flow="column"))
container_three = widgets.Box([
    Label("Table View")],
    layout=nav_display_style)
container_four = widgets.Box([
    Label("Logs", layout=output_header_style),logger_out],
    layout=nav_display_style)
container_five = widgets.Box([interactive_tabs])

nav_container = widgets.Box(
    [navigation,out],
    layout=widgets.Layout(
    display="flex",
    flex_flow='row'
    ))
main_container = widgets.Box(
    [connection_inputs,nav_container],
    layout=widgets.Layout(
    display="flex",
    flex_flow="column",
    marging="auto"
    )
)

# display(main_container)


def nav_btn_click(b):
    show_display = b.description
    out.clear_output()
    if show_display == 'Exp Details':
        with out:
            display(container_one)
    elif show_display == "Plot View":
        with out:
            display_plot()
            display(container_two)
    elif show_display == "Table View":
         with out: 
            display(container_three)
    elif show_display == "Logs":
         with out: 
            display(container_four)
    elif show_display == "Interactive":
         with out: 
            display(container_five)

            
exp_details.on_click(nav_btn_click)    
plot_view.on_click(nav_btn_click)
table_view.on_click(nav_btn_click)
logs.on_click(nav_btn_click)
interactive.on_click(nav_btn_click)


#------ View 1 ------            
uploader = widgets.FileUpload(
    description="Connect to Database",accept=".db", multiple=False, layout=Layout(margin="15px 60px"))

def start_server():
    global strat
    global server
    with error_output:
        try:
#             with error_output:
#                 try: 
            server = AEPsychServer(database_path=database_path)
            strat = server.get_strat_from_replay()
            if display_plot() == True:
                home_view_container.layout.display = 'none' 
                main_container.layout.display = 'inline'
            else:
                error_output.clear_output()
                if strat != None:
                    try: 
                        plot_strat(
                            strat,
                            xlabel=xlabel.value,
                            ylabel=ylabel.value,
                            no_label=no_label.value,
                            yes_label=yes_label.value,
                            cred_level=cred_level.value,
                            target_level=target_level.value,
                        )
                    except RuntimeError as err: 
                        error_output.clear_output()
                        with error_output:
                            print("Unable to connect to server. \n")
                            print(err)
                    except AssertionError as err: 
                        print(err)

                display(main_container)
        except sqlalchemy.exc.DatabaseError as err:
                    error_output.clear_output()
                    with error_output:
                        print("Unable to connect to server. \n")
                        print(err)
        except Exception as e:
            print("unable to connect to server...")
            print(e)
#             exc_type, exc_obj, exc_tb = sys.exc_info()
#             fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#             print(exc_type, fname, exc_tb.tb_lineno)
            

def upload():
    global server
    global strat
    global database_path
    error_output.clear_output() 
    with error_output:     
        error_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else:
            for file_name, file in uploader.value.items():
                current_path = os.getcwd()
                external_target_path = current_path + "/databases"
                completeName = None 
                
                if os.path.isdir(external_target_path):
                    completeName = os.path.join(external_target_path, file_name)
                else:
                    print("File upload unsuccessful")

                with open(completeName, "wb") as f:
                    f.write(file["content"])
                    if completeName != None:
                        database_path = completeName
                    start_server()
                    f.close()
                    

def uploader_on_change(change): 
    error_output.clear_output()
    with error_output:
        try: 
            upload()
#             start_server()
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)

uploader.observe(uploader_on_change, names="value")

error_output = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))
build_experiment = widgets.Button(
    description='Build Experiment',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    layout=Layout(width='auto', height="auto", margin="15px 60px"))

home_view_container = widgets.Box([
    Label("Build a new experiment, or load data from an old one"), 
    widgets.HBox([build_experiment, uploader], 
                layout=widgets.Layout(
                display="flex",
                justify_content="center",
                margin="30px 0",
                align_items="center"
                )),
    error_output,],
    layout=Layout(
    display="flex",
    flex_flow='column',
    width="100%",
    height="100%",
    margin="60px 0",
    justify_content='center',
    align_items="center",))

# display(home_view_container)
# home_view_container.layout.display = 'none'

wrapper_container = widgets.Box([header,main_container,home_view_container],
                               layout=Layout(display="flex",flex_flow="column"))
main_container.layout.display = 'none'
display(wrapper_container)

def build():
    error_output.clear_output()
    with error_output: 
        print("501 Not Implemented \n * Functionality has Not been Implemented")
#         home_view_container.layout.display = 'none'

    
def connect(): 
    error_output.clear_output()
    with error_output:
        start_server()
        
    
def build_experiment_clicked(b):
    build()

    
def connect_to_database_clicked(b):
    connect()

    
build_experiment.on_click(build_experiment_clicked)

Box(children=(HTML(value='\n<div class="nav" style=" \nmargin: 0px;\nbackground-color: #F2F2F2;\nfont-weight: …